In [1]:
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from bson import ObjectId
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB

In [2]:
ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = './ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)
cursor = conn.cursor()

In [6]:
conn.rollback()
from datetime import datetime

# 기관 리스트 정의
institutions = [
    "NewEdge Finance", "MYCREDIT INVESTMENTS LIMITED", "Carbon Microfinance Bank Limited",
    "Aella Financial Solution Ltd", "MyCredit Invest", "BLUERIDGE MICROFINANCE BANK LIMITED",
    "Access Bank Plc Lagos", "Arve Limited", "Guaranty Trust Bank Plc",
    "Fairmoney Microfinance Bank Limited", "Credit Direct Ltd Lagos",
    "RenMoney MFB Nigeria Limited  Lagos", "Zedvance Limited Lagos",
    "ALTRACRED FINANCE INVESTMENT LIMITED", "MyCredit Investments Limited",
    "NewEdge Finance Limited", "AELLA FINANCIAL SOLUTIONS LIMITED",
    "CashXpress Nigeria Limited", "BetterLending Company Limited",
    "MIM Finance Company Limited", "CARBON FINANCE AND INVESTMENT LIMITED",
    "BRANCH INTERNATIONAL FINANCIAL SERVICE LIMITED", "Access Bank Plc",
    "Branch International Financial Services Limited", "RENMONEY MICROFINANCE BANK LIMITED",
    "FAIRMONEY MICROFINANCE BANK LIMITED", "First City Monument Bank Ltd Lagos",
    "Zedvance Limited", "Polaris Bank Limited Lagos", "Crednet Technologies Limited (CredPal)"
]

# 결과를 저장할 리스트
all_results = []

# 각 기관별로 연도별 부실률 계산
for institution in institutions:
    result_row = {'전체기관': institution}
    
    # 각 연도별 부실률 계산
    for year in range(2015, 2025):
        qry = """
            SELECT 
                TRUNC(
                    CAST(
                        CAST(SUM(CASE WHEN ch."status" = 'lost' THEN 1 ELSE 0 END) AS FLOAT) / 
                        COUNT(ch."createdAt")
                    AS NUMERIC
                    ), 
                    2
                ) as default_rate
            FROM credit_history ch
            WHERE ch."openedAt" >= %s 
            AND ch."openedAt" < %s
            AND ch."institutionName" = %s
        """
        
        start_date = f'{year}-01-01'
        end_date = f'{year+1}-01-01'
        
        cursor.execute(qry, (start_date, end_date, institution))
        default_rate = cursor.fetchone()[0]
        
        # None인 경우 0으로 처리
        result_row[f'{year} 부실률'] = default_rate if default_rate is not None else 0
    
    all_results.append(result_row)

# DataFrame 생성
columns = ['전체기관'] + [f'{year} 부실률' for year in range(2015, 2025)]
df = pd.DataFrame(all_results, columns=columns)

# CSV 저장
df.to_csv('연도별_기관별_부실률.csv', index=False, encoding='utf-8-sig')

# 결과 확인
print(df.head())

                               전체기관 2015 부실률 2016 부실률 2017 부실률 2018 부실률  \
0                   NewEdge Finance        0        0        0        0   
1      MYCREDIT INVESTMENTS LIMITED        0        0     0.00     0.00   
2  Carbon Microfinance Bank Limited        0     0.20     0.00     0.04   
3      Aella Financial Solution Ltd        0        0     0.00     0.00   
4                   MyCredit Invest        0        0     0.00     0.50   

  2019 부실률 2020 부실률 2021 부실률 2022 부실률 2023 부실률 2024 부실률  
0     0.12     0.09     0.12     0.08     0.22     0.04  
1     0.08     0.08     0.17     0.71     0.91        0  
2     0.05     0.03     0.01     0.04     0.00     1.00  
3     0.00     0.00     0.00     0.00     0.00     0.00  
4     0.26     0.39     0.38     0.02     0.03        0  
